In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from timeit import default_timer as timer
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

In [9]:
batch_size = 64
num_classes = 2
learnin_rate = 0.0005
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
print(torch.cuda.is_available())

print(torch.__version__)

True
2.1.2+cu121


In [11]:
dataset = torch.load('images_perso.pth')
labels = torch.load('labels_perso.pth')

In [12]:
class handDataset(torch.utils.data.Dataset):

    def __init__(self):
        self.x = dataset
        self.y = labels
        self.n_samples = len(dataset)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [13]:
generator = torch.Generator().manual_seed(42)

train_size = int(len(dataset) * .7)
test_size = len(dataset) - train_size

hand_dataset = handDataset()
train_dataset, test_dataset = torch.utils.data.random_split(hand_dataset, [train_size, test_size], generator=generator)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [14]:
class FF_model(nn.Module):
    def __init__(self, num_classes):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*64*3,500) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(500, num_classes)  
    
    def forward(self, x):
        out = self.flatten(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [15]:
class CNN_model(nn.Module):

    def __init__(self, num_classes):
        super(CNN_model, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(14*30*64,128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.max_pool1(out)
        out = self.conv_layer2(out)
        out = self.max_pool2(out)
        out = self.flatten(out)
        #896x14 and 12544x128)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)

        return out


In [16]:
cnn_model = CNN_model(num_classes).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(cnn_model.parameters(), lr=learnin_rate, weight_decay=0.005, momentum=0.9)

total_step = len(train_loader)

In [17]:
for epoch in range(num_epochs):

    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images=images.to(device)
        labels=labels.to(device)
        
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

5it [00:00, 12.17it/s]


Epoch [1/10], Loss: 0.5951


5it [00:00, 84.62it/s]


Epoch [2/10], Loss: 0.5887


5it [00:00, 86.25it/s]


...

Epoch [8/10], Loss: 0.3390


5it [00:00, 96.27it/s]


Epoch [9/10], Loss: 0.2753


5it [00:00, 89.22it/s]


Epoch [10/10], Loss: 0.2536


In [18]:
with torch.no_grad():
    correct = 0
    total = 0
    start = timer()
    for images, labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        labels_class=torch.max(labels,1)
        print(labels_class)
        for i,sample in enumerate(labels_class[1]):
            print(sample)
            print(predicted[i])
            if sample==predicted[i]:
                correct += 1
    end=timer()
    
    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))
    print("TIme of inference =" + str((end-start)/total) +"s")



torch.return_types.max(
values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0'),
indices=tensor([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1], device='cuda:0'))
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, devi

In [19]:
from PIL import Image
tr = transforms.Compose([
        transforms.Resize((128, 64)),
        transforms.ToTensor(),
        #transforms.RandomHorizontalFlip(),  
        #transforms.RandomRotation(90),  
        #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  
        transforms.Normalize(mean=[0], std=[0.5])  # Normalizing mean and std values
    ])
image = Image.open("./WIN_20240111_10_48_03_Pro (2).jpg").convert('L')
image.show()
image_tensor = tr(image)
image_tensor=image_tensor.to(device)
image_batch = image_tensor.unsqueeze(0)
predicted=cnn_model(image_batch)
print(predicted)


tensor([[-0.1293, -0.0484]], device='cuda:0', grad_fn=<AddmmBackward0>)


## affichage des résultats

In [20]:
%load_ext tensorboard
%tensorboard --logdir=runs

Reusing TensorBoard on port 6006 (pid 4812), started 2:10:10 ago. (Use '!kill 4812' to kill it.)

In [21]:
import cv2 as cv
import numpy as np
from PIL import Image


img_path = "./WIN_20240111_10_48_03_Pro (2).jpg"
img = cv.imread(img_path)
fgbg = cv.createBackgroundSubtractorMOG2()
fgmask = fgbg.apply(img)
array_im=cv.cvtColor(fgmask, cv.COLOR_BGR2RGB)
pil_image=Image.fromarray(array_im)
pil_image.show()
hsvim = cv.cvtColor(img, cv.COLOR_BGR2HSV) 
lower = np.array([0, 48, 80], dtype = "uint8") 
upper = np.array([50, 255, 255], dtype = "uint8") 
skinRegionHSV = cv.inRange (hsvim, lower, upper) 
blurred= cv.blur (skinRegionHSV, (2,2))
ret, thresh = cv.threshold (blurred, 0,255, cv. THRESH_BINARY) 
array_im=cv.cvtColor(thresh, cv.COLOR_BGR2RGB)
pil_image=Image.fromarray(array_im)
pil_image.show()



## Sauvegarde du modèle

In [23]:
model_scripted = torch.jit.script(cnn_model) # Export to TorchScript
model_scripted.save('sauvegarde.pt') # Save